In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import datetime
import calendar
from scipy import sparse
from pandas.api.types import CategoricalDtype

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
cd D:\Course\SBRS

D:\Course\SBRS


In [5]:
# ctg_file = 'datasets/retailrocket/category_tree.csv'
events_file = 'datasets/retailrocket/events.csv'
# item_prop_1 = 'datasets/retailrocket/item_properties_part1.csv'
# item_prop_2 = 'datasets/retailrocket/item_properties_part2.csv'

In [6]:
# ctg_df = pd.read_csv(ctg_file)
events_df = pd.read_csv(events_file)
# item_prop_1_df = pd.read_csv(item_prop_1)
# item_prop_2_df = pd.read_csv(item_prop_2)


In [9]:
events_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [11]:
events_actions = {
    'view': 1,
    'addtocart':2,
    'transaction':3
}

In [12]:
events_df.event = [events_actions[item] for item in events_df.event]

In [13]:
events_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,1,355908,NaN
1,1433224214164,992329,1,248676,NaN
2,1433221999827,111016,1,318965,NaN
3,1433221955914,483717,1,253185,NaN
4,1433221337106,951259,1,367447,NaN


In [14]:
events_df['event_date'] = pd.to_datetime(events_df['timestamp'])
events_df = events_df.drop('timestamp', axis=1)


In [15]:
events_df['date'] = events_df.event_date.dt.date
events_df['time'] = events_df.event_date.dt.time
events_df['year'] = events_df.event_date.dt.year
events_df['month'] = events_df.event_date.dt.month
events_df['day'] = events_df.event_date.dt.day
events_df['hour'] = events_df.event_date.dt.hour
events_df = events_df.drop(['event_date','time','date'], axis=1)

In [16]:
events_df.head()

,visitorid,event,itemid,transactionid,year,month,day,hour
0,257597,1,355908,NaN,1970,1,1,0
1,992329,1,248676,NaN,1970,1,1,0
2,111016,1,318965,NaN,1970,1,1,0
3,483717,1,253185,NaN,1970,1,1,0
4,951259,1,367447,NaN,1970,1,1,0


In [17]:
events_df.transactionid.fillna(-1, inplace=True)

In [18]:
events_df.head()

,visitorid,event,itemid,transactionid,year,month,day,hour
0,257597,1,355908,-1.0,1970,1,1,0
1,992329,1,248676,-1.0,1970,1,1,0
2,111016,1,318965,-1.0,1970,1,1,0
3,483717,1,253185,-1.0,1970,1,1,0
4,951259,1,367447,-1.0,1970,1,1,0


In [23]:
users = events_df.visitorid.unique()
items = events_df.itemid.unique()
shape = (len(users),len(items))

In [26]:
# Categorical coding
users_cat = CategoricalDtype(categories=sorted(users), ordered=True)
items_cat = CategoricalDtype(categories=sorted(items), ordered=True)
users_index = events_df.visitorid.astype(users_cat).cat.codes
items_index = events_df.itemid.astype(items_cat).cat.codes

In [27]:
# To csr
coo = sparse.coo_matrix((events_df.event, (users_index, items_index)), shape=shape)
csr = coo.tocsr()